In [1]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V

## pytorch反向传播的方法：

首先按照顺序计算variable得到计算图，之后在得到节点loss（不一定要是祖先节点，只要可导哪怕是叶子节点都可以）之后，采用loss.backward()方法求出每个variable的导数（注意若loss不是标量，要采取loss.backward(t.ones(loss.size()))）的方法进行反向传播原因见
https://zhuanlan.zhihu.com/p/29923090

注意：对于需要求导的variable，需要设置requires_grad = True。

eg:a = V(t.ones(3,4), requires_grad = True) 

若c是a的双亲结点，即使没有指定c需要求导，但c依赖于a，而a需要求导，
因此c的requires_grad属性会自动设为True。

但是，c不是叶子节点（是不是叶子节点可以从图中或者运行c.is_leaf进行判断），其梯度是用来计算a的梯度。所以，c的梯度计算完之后即被释放。



variable.grad_fn.next_function可能就是返回该节点到下一节点的反向传播函数的函数类型，若下一节点不需求导（requires_grad=false）.返回值为none

计算w的梯度的时候，需要用到x的数值(${\partial y\over \partial w} = x $)，这些数值在前向过程中会保存成buffer，在计算完梯度之后会自动清空。为了能够多次反向传播需要指定`retain_graph`来保留这些buffer。

变量的`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都是True。这其实很好理解，对于$ \textbf{x}\to \textbf{y} \to \textbf{z}$，x.requires_grad = True，当需要计算$\partial z \over \partial x$时，根据链式法则，$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$，自然也需要求$ \frac{\partial z}{\partial y}$，所以y.requires_grad会被自动标为True. 

`volatile=True`是另外一个很重要的标识，它能够将所有依赖于它的节点全部都设为`volatile=True`，其优先级比`requires_grad=True`高。`volatile=True`的节点不会求导，即使`requires_grad=True`，也无法进行反向传播。对于不需要反向传播的情景（如inference，即测试推理时），该参数可实现一定程度的速度提升，并节省约一半显存，因其不需要分配空间计算梯度。

在反向传播过程中非叶子节点的导数计算完之后即被清空。若想查看这些变量的梯度，有两种方法：
- 使用autograd.grad函数
- 使用hook

推荐使用`hook`方法，但是在实际使用中应尽量避免修改grad的值。hook相关讲解：https://www.zhihu.com/question/61044004

## torch.autograd.function自定义函数
这里我还不是太懂，不过大致知道以后利用torch.autograd.function自定义函数应该怎么做了。

- 首先，把书中的例子复制粘贴一下
- 之后，按照函数计算方式与求导方式修改自定义的function类
- 然后，采用Function.apply(variable)来调用自定义的函数（）
- 最后，再执行反向传播的时候有两种方式一种是按之前一样实现loss.backward(),另一种是采用z.grad_fn.apply(V(t.ones(1))) 这种方式可以保留一些原本输入的是tensor的数据的求导结果
- 最最后，建议在自己实现完一个函数之后用梯度检查的方法看一下自己实现的函数对不对，方法见下面

在实现了自己的Function之后，还可以使用gradcheck函数来检测实现是否正确。gradcheck通过数值逼近来计算梯度，可能具有一定的误差，通过控制eps的大小可以控制容忍的误差。

注意以下几点：

- 自定义的Function需要继承autograd.Function，没有构造函数`__init__`，forward和backward函数都是静态方法
- forward函数的输入和输出都是Tensor，backward函数的输入和输出都是Variable
- backward函数的输出和forward函数的输入一一对应，backward函数的输入和forward函数的输出一一对应
- backward函数的grad_output参数即t.autograd.backward中的`grad_variables`
- 如果某一个输入不需要求导，直接返回None，如forward中的输入参数x_requires_grad显然无法对它求导，直接返回None即可
- 反向传播可能需要利用前向传播的某些中间结果，需要进行保存，否则前向传播结束后这些对象即被释放

采用数值逼近方式检验计算梯度的公式对不对,其中Sigmoid.apply可以换成任何你实现的函数的名称.apply（似乎不是自己写的函数用gradcheck直接用函数名就可以了）而(test_input,)中test_input可以换成任何你用于测试函数自动求导是否OK的variable,不过，格式要写成(test_input,)类似的形式，eps=1e-3这部分应该是最大容许的误差（就是与你自动求导的梯度与导数定义公式算出来的梯度的最大容许的差距）

In [2]:

test_input = V(t.randn(3,4), requires_grad=True)
t.autograd.gradcheck(Sigmoid.apply, (test_input,), eps=1e-3)


 1
[torch.FloatTensor of size 1]